In [ ]:
! pip install pytorch-lightning~=1.5.0
! pip install datasets~=2.1.0
! pip install transformers
! pip install torchmetrics~=0.7.0
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline
from datasets import load_dataset
from datasets import load_metric
import torch
import torch.nn as nn
from transformers import RobertaTokenizerFast,RobertaForSequenceClassification, AdamW, TrainingArguments, Trainer,AutoModelForSequenceClassification
dataset = load_dataset("McGill-NLP/FaithDial")


Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3539 [00:00<?, ? examples/s]

Generating test_random_split split:   0%|          | 0/1716 [00:00<?, ? examples/s]

Generating test_topic_split split:   0%|          | 0/1823 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3417 [00:00<?, ? examples/s]

Generating valid_random_split split:   0%|          | 0/1666 [00:00<?, ? examples/s]

Generating valid_topic_split split:   0%|          | 0/1751 [00:00<?, ? examples/s]

Dataset faith_dial_dataset downloaded and prepared to /root/.cache/huggingface/datasets/McGill-NLP___faith_dial_dataset/plain_text/1.0.0/70568c8ab3bbc83b603bce58fa593ab27e7f0d0cde51034e1c2073ff3e14189a. Subsequent calls will reuse this data.


  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
updated_dataset = dataset.map(lambda examples: {"tag": "Entailment"  if (examples["original_response"] is None and "Hallucination" not in examples["BEGIN"]) else "Hallucination"})

  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

In [ ]:
ud = updated_dataset.map(lambda examples: {"original_response": examples["response"]  if examples["original_response"] == None else examples["original_response"]})


  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

In [ ]:
sample = ud
sample

DatasetDict({
    test: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM', 'tag'],
        num_rows: 3539
    })
    test_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM', 'tag'],
        num_rows: 1716
    })
    test_topic_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM', 'tag'],
        num_rows: 1823
    })
    train: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM', 'tag'],
        num_rows: 18357
    })
    validation: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 'VRM', 'tag'],
        num_rows: 3417
    })
    valid_random_split: Dataset({
        features: ['dialog_idx', 'response', 'original_response', 'history', 'knowledge', 'BEGIN', 

In [ ]:
sample_dataset = sample.map(lambda examples: {"tag": "Entailment"})

  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

In [ ]:
sd = sample_dataset.map(lambda examples: {"original_response": examples["response"]})


  0%|          | 0/3539 [00:00<?, ?ex/s]

  0%|          | 0/1716 [00:00<?, ?ex/s]

  0%|          | 0/1823 [00:00<?, ?ex/s]

  0%|          | 0/18357 [00:00<?, ?ex/s]

  0%|          | 0/3417 [00:00<?, ?ex/s]

  0%|          | 0/1666 [00:00<?, ?ex/s]

  0%|          | 0/1751 [00:00<?, ?ex/s]

In [ ]:
from datasets.dataset_dict import DatasetDict
overall = DatasetDict()

In [ ]:
from datasets import concatenate_datasets
assert ud["train"].features.type == sd["train"].features.type

overall["train"] = concatenate_datasets([sd["train"],ud["train"]])
overall["test"] = concatenate_datasets([sd["test"],ud["test"]])
overall["validation"] = concatenate_datasets([sd["validation"],ud["validation"]])


In [ ]:
overall2 = overall.map(lambda data:{"history":""+(data["knowledge"]) if data["history"] is None else data["history"][-1]+"</s>"+data["knowledge"]})


  0%|          | 0/36714 [00:00<?, ?ex/s]

  0%|          | 0/7078 [00:00<?, ?ex/s]

  0%|          | 0/6834 [00:00<?, ?ex/s]

In [ ]:

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 1024)

In [ ]:
def preprocess_function(examples):
    encoding=tokenizer(ew, truncation=True)
    lable= 1 if examples['tag'] == "Hallucination" else 0
    encoding["labels"]=lable
    
    return encoding
encoded_dataset = overall2.map(preprocess_function,remove_columns=ud['train'].column_names)

  0%|          | 0/36714 [00:00<?, ?ex/s]

  0%|          | 0/7078 [00:00<?, ?ex/s]

  0%|          | 0/6834 [00:00<?, ?ex/s]

In [ ]:
id2label = { 1: 'Hallucination', 0: 'Entailment'}
label2id = {'Hallucination': 1, 'Entailment': 0}

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(label2id),label2id=label2id,id2label=id2label)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
import evaluate
import numpy as np
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    adafactor= False,
adam_epsilon= 1.0e-08,
    logging_dir='./logs',
    evaluation_strategy='steps',
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate(encoded_dataset['test'])
print(results)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.474600,0.537199,0.829382,0.801024
1000,0.373300,0.370692,0.844308,0.790798
1500,0.344500,0.342769,0.860697,0.837709
2000,0.337800,0.335975,0.853819,0.817667
2500,0.308300,0.378688,0.853819,0.812535
3000,0.263200,0.472608,0.857477,0.828219
3500,0.268600,0.377830,0.869037,0.841789
4000,0.240100,0.501003,0.859819,0.843719
4500,0.253900,0.428627,0.865525,0.840811
5000,0.193200,0.514543,0.869622,0.847770


{'eval_loss': 0.48015010356903076, 'eval_accuracy': 0.8741169821983611, 'eval_f1': 0.8499242041435068, 'eval_runtime': 49.7022, 'eval_samples_per_second': 142.408, 'eval_steps_per_second': 2.233, 'epoch': 3.0}


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!pip install shutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
  Created wheel for shutils: filename=shutils-0.1.0-py3-none-any.whl size=3516 sha256=1153749e349d74e6fb27a1d2dff8f0568c88eab77f014fb37d16decafdf69509
  Stored in directory: /root/.cache/pip/wheels/62/63/04/81e549bdb44792d8b62938cffc3bd00a34addabe1da3693db8
Successfully built shutils


In [ ]:
import os

os.makedirs('/content/gdrive/MyDrive/NLP/combined')

In [ ]:
import shutil
for filename in os.listdir('/content/results/checkpoint-6500'):
  shutil.move(os.path.join('/content/results/checkpoint-6500',filename),os.path.join('/content/gdrive/MyDrive/NLP/combined',filename) )
  # print(filename)

In [ ]:
results_pred = trainer.predict(encoded_dataset['test'])


***** Running Prediction *****
  Num examples = 3539
  Batch size = 64


In [ ]:
logits = results_pred.predictions


In [ ]:
import tensorflow as tf

In [ ]:
transformed = tf.nn.softmax(logits)

In [ ]:
transformed[:5]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.12654774, 0.8734522 ],
       [0.12654829, 0.87345165],
       [0.12654889, 0.8734511 ],
       [0.12654853, 0.8734515 ],
       [0.12654796, 0.873452  ]], dtype=float32)>

In [ ]:
ground_truth = (encoded_dataset["test"]['labels'])

In [ ]:
transformed_labels = np.argmax(transformed, axis=1)

In [ ]:
# Load f1 metric
metric_f1 = evaluate.load("f1")

# Compute f1 metric
f1 = metric_f1.compute(predictions=transformed_labels, references=ground_truth)

# # Load recall metric
metric_recall = evaluate.load("recall")


metric_accuracy = evaluate.load("accuracy")
accu = metric_accuracy.compute(predictions=transformed_labels, references=ground_truth)

# # Compute recall metric
 = metric_recall.compute(predictions=transformed_labels, references=ground_truth)